In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import os, random

In [2]:
# Specify the directories and layer inresults_dirtion
results_dir = '.'

In [3]:
target_img_info = pd.read_csv((os.path.join(results_dir, 'selected_imgs_sEEG.csv')))

In [4]:
fillers_images_dir = os.path.join(results_dir, 'study_questions_filler')
# 获取filler图片列表
filler_image_files = []
if os.path.exists(fillers_images_dir):
    for file in os.listdir(fillers_images_dir):
        if file.lower().endswith('.jpg'):
            filler_image_files.append(file)
else:
    raise FileNotFoundError(f"Filler图片目录不存在: {fillers_images_dir}")

print(f"找到 {len(filler_image_files)} 张filler图片")
for file in filler_image_files:
    print(f"  - {file}")

找到 20 张filler图片
  - nonhuman9.jpg
  - nonhuman8.jpg
  - human4.jpg
  - human1.jpg
  - human2.jpg
  - human3.jpg
  - nonhuman1.jpg
  - nonhuman16.jpg
  - nonhuman14.jpg
  - nonhuman3.jpg
  - nonhuman2.jpg
  - nonhuman15.jpg
  - nonhuman11.jpg
  - nonhuman6.jpg
  - nonhuman7.jpg
  - nonhuman10.jpg
  - nonhuman12.jpg
  - nonhuman5.jpg
  - nonhuman4.jpg
  - nonhuman13.jpg


In [5]:
# Specify the parameters
n_blocks = 4
n_categories = 4
study_imgs_per_category_per_block = 30  # 每个block中每个类别在study阶段的图片数
test_old_imgs_per_category_per_block = 15  # 每个block中每个类别在test阶段的旧图片数
test_new_imgs_per_category_per_block = 15  # 每个block中每个类别在test阶段的新图片数
n_fillers_per_block = 5


In [6]:
# Sort the images into different groups
category_1 = target_img_info[(target_img_info['RE group']=='Large')&
                            (target_img_info['Dist group']=='Small')].reset_index(drop=True)
category_2 = target_img_info[(target_img_info['RE group']=='Large')&
                            (target_img_info['Dist group']=='Large')].reset_index(drop=True)
category_3 = target_img_info[(target_img_info['RE group']=='Small')&
                            (target_img_info['Dist group']=='Small')].reset_index(drop=True)
category_4 = target_img_info[(target_img_info['RE group']=='Small')&
                            (target_img_info['Dist group']=='Large')].reset_index(drop=True)
img_category_dict = {
    1: category_1,
    2: category_2,
    3: category_3,
    4: category_4
}

In [7]:
category_1

,Unnamed: 0,Index,Image type,Image index,Distinctiveness,Reconstruction error,Distinctiveness rank,RE rank,RE group,Dist group,Rank diff,Rank sum
0,0,Target-3,Target,3,821.592265,0.070374,2604.0,9215.0,Large,Small,NaN,NaN
1,1,Target-8,Target,8,819.107269,0.073270,2578.0,9423.0,Large,Small,NaN,NaN
2,2,Target-16,Target,16,633.874588,0.063714,1061.0,8528.0,Large,Small,NaN,NaN
3,3,Target-113,Target,113,744.857872,0.061535,1847.0,8231.0,Large,Small,NaN,NaN
4,4,Target-121,Target,121,679.619208,0.064591,1336.0,8648.0,Large,Small,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
190,190,Filler-7648,Filler,7648,737.341632,0.058792,1784.0,7846.0,Large,Small,NaN,NaN
191,191,Filler-7802,Filler,7802,656.543958,0.075082,1198.0,9529.0,Large,Small,NaN,NaN
192,192,Filler-7963,Filler,7963,715.851746,0.059012,1599.0,7885.0,Large,Small,NaN,NaN
193,193,Filler-8013,Filler,8013,736.082692,0.072915,1770.0,9396.0,Large,Small,NaN,NaN


In [8]:
def generate_study_sequence(num_categories=4, images_per_category=30, num_fillers=5, seed=None):
    """生成study阶段的block序列"""
    if seed is not None:
        random.seed(seed)

    # 生成所有target trial
    target_trials = []
    for category in range(1, num_categories + 1):
        for img_idx in range(1, images_per_category + 1):
            target_trials.append({
                'label': f'C{category}_S{img_idx}',
                'type': 'target',
                'has_question': False
            })

    # 生成filler trial
    filler_trials = []
    for filler_idx in range(1, num_fillers + 1):
        filler_trials.append({
            'label': f'F{filler_idx}',
            'type': 'filler',
            'has_question': True
        })

    # 合并并均匀分布filler图片
    all_trials = []
    target_per_section = len(target_trials) // (num_fillers + 1)

    for i in range(num_fillers + 1):
        start_idx = i * target_per_section
        end_idx = (i + 1) * target_per_section if i < num_fillers else len(target_trials)
        all_trials.extend(target_trials[start_idx:end_idx])

        if i < num_fillers:
            all_trials.append(filler_trials[i])

    # 随机打乱每个section内的target图片顺序，但保持filler位置
    final_trials = []
    current_section = []
    for trial in all_trials:
        if trial['type'] == 'filler':
            random.shuffle(current_section)
            final_trials.extend(current_section)
            final_trials.append(trial)
            current_section = []
        else:
            current_section.append(trial)

    if current_section:
        random.shuffle(current_section)
        final_trials.extend(current_section)

    # 格式化输出
    sequence = []
    for trial_num, trial_info in enumerate(final_trials, 1):
        if trial_info['type'] == 'target':
            category = int(trial_info['label'].split('_')[0][1:])
        else:
            category = None

        sequence.append({
            'trial_num': trial_num,
            'image_label': trial_info['label'],
            'category': category,
            'type': 'study',
            'image_type': trial_info['type'],
            'has_question': trial_info['has_question']
        })

    return sequence


In [9]:
def generate_test_sequence(old_images, new_images, seed=None):
    """生成test阶段的序列"""
    if seed is not None:
        random.seed(seed)

    # 合并旧图片和新图片
    all_trials = []

    # 添加旧图片
    for img_label in old_images:
        all_trials.append({
            'label': img_label,
            'type': 'old'
        })

    # 添加新图片
    for category, img_labels in new_images.items():
        for img_label in img_labels:
            all_trials.append({
                'label': img_label,
                'type': 'new'
            })

    # 随机打乱顺序
    random.shuffle(all_trials)

    # 格式化输出
    sequence = []
    for trial_num, trial_info in enumerate(all_trials, 1):
        label_parts = trial_info['label'].split('_')
        category = int(label_parts[0][1:])
        sequence.append({
            'trial_num': trial_num,
            'image_label': trial_info['label'],
            'category': category,
            'type': trial_info['type']
        })

    return sequence

In [10]:
NB_dir   = os.getcwd()
PROJECT  = os.path.dirname(os.path.dirname(NB_dir))
Exp_dir  = os.path.join(PROJECT, 'Experiment/')

os.makedirs(Exp_dir, exist_ok=True)
print("experiment_design directory：", Exp_dir)

experiment_design directory： /Users/xuezhang/Documents/SKKU/ReconMem_new_design/Experiment/


In [12]:
for sub in range(1, 51):
    sub_dir = os.path.join(Exp_dir, f"sub-{sub:02d}")
    if not os.path.isdir(sub_dir):
        os.makedirs(sub_dir, exist_ok=True)

    print(f"为被试 {sub} 生成实验序列...")

    # 为每个被试随机选择所有block需要的图片
    all_selected_images = {}  # 记录每个类别选择的图片

    for category in range(1, n_categories + 1):
        category_df = img_category_dict[category]

        # 计算每个类别需要的总图片数
        total_imgs_needed = n_blocks * (study_imgs_per_category_per_block + test_new_imgs_per_category_per_block)

        # 随机选择所需的所有图片
        selected_indices = random.sample(range(len(category_df)), total_imgs_needed)
        all_selected_images[category] = category_df.iloc[selected_indices].reset_index(drop=True)

    # 为每个block生成序列
    for block in range(1, n_blocks + 1):
        print(f"  生成block {block}...")

        # 计算当前block使用的图片范围
        start_idx = (block - 1) * (study_imgs_per_category_per_block + test_new_imgs_per_category_per_block)
        study_end_idx = start_idx + study_imgs_per_category_per_block
        test_new_end_idx = study_end_idx + test_new_imgs_per_category_per_block

        # 为当前block选择图片
        block_study_images = {}  # 当前block的study图片
        block_test_new_images = {}  # 当前block的test新图片
        block_study_labels = {}  # study图片的标签
        block_test_new_labels = {}  # test新图片的标签

        for category in range(1, n_categories + 1):
            # study图片
            study_imgs = all_selected_images[category].iloc[start_idx:study_end_idx]
            block_study_images[category] = study_imgs
            block_study_labels[category] = [f'C{category}_S{i+1}' for i in range(len(study_imgs))]

            # test新图片
            test_new_imgs = all_selected_images[category].iloc[study_end_idx:test_new_end_idx]
            block_test_new_images[category] = test_new_imgs
            block_test_new_labels[category] = [f'C{category}_N{i+1}' for i in range(len(test_new_imgs))]

         # 为当前block选择filler图片（所有被试使用相同的filler图片）
        # 使用固定的随机种子确保所有被试使用相同的filler图片
        filler_seed = block * 100  # 只基于block编号，不基于被试编号
        random.seed(filler_seed)
        selected_filler_files = random.sample(filler_image_files, n_fillers_per_block)
        random.seed()  # 重置随机种子

        print(f"    Block {block} 使用的filler图片: {selected_filler_files}")

        # 生成study序列
        study_seed = sub * 1000 + block * 10
        study_sequence = generate_study_sequence(
            num_categories=n_categories,
            images_per_category=study_imgs_per_category_per_block,
            num_fillers=n_fillers_per_block,
            seed=study_seed
        )

        # 为study序列分配实际图片路径
        study_image_lookup = {}
        for category in range(1, n_categories + 1):
            for label, (idx, img_row) in zip(block_study_labels[category], block_study_images[category].iterrows()):
                study_image_lookup[label] = img_row['Index']

        # 分配filler图片路径
        for trial in study_sequence:
            if trial['image_type'] == 'target':
                trial['Image_Index'] = study_image_lookup.get(trial['image_label'], None)
                trial['Image_Name'] = None  # target图片没有单独的文件名
            else:  # filler图片
                filler_idx = int(trial['image_label'][1:]) - 1
                trial['Image_Index'] = os.path.join('fillers_images', selected_filler_files[filler_idx])
                trial['Image_Name'] = selected_filler_files[filler_idx]  # 记录图片文件名
                trial['Filler_Index'] = filler_idx + 1

        # 保存study部分
        study_df = pd.DataFrame(study_sequence)
        study_df['Block'] = block
        study_df['Part'] = 'study'

        study_filename = os.path.join(sub_dir, f"block{block:02d}_study.csv")
        study_df.to_csv(study_filename, index=False)

        # 生成test序列
        # 从study图片中随机选择一半作为test的旧图片
        test_old_labels = {}
        for category in range(1, n_categories + 1):
            old_indices = random.sample(range(study_imgs_per_category_per_block), test_old_imgs_per_category_per_block)
            test_old_labels[category] = [block_study_labels[category][i] for i in old_indices]

        # 将所有旧图片标签合并
        all_old_labels = []
        for category in range(1, n_categories + 1):
            all_old_labels.extend(test_old_labels[category])

        test_seed = sub * 1000 + block * 10 + 1
        test_sequence = generate_test_sequence(
            old_images=all_old_labels,
            new_images=block_test_new_labels,
            seed=test_seed
        )

        # 为test序列分配实际图片路径
        test_image_lookup = {}

        # 添加旧图片映射
        for category in range(1, n_categories + 1):
            for label in test_old_labels[category]:
                # 从study图片中查找对应的图片路径
                img_idx = int(label.split('S')[1]) - 1
                test_image_lookup[label] = block_study_images[category].iloc[img_idx]['Index']

        # 添加新图片映射
        for category in range(1, n_categories + 1):
            for i, label in enumerate(block_test_new_labels[category]):
                test_image_lookup[label] = block_test_new_images[category].iloc[i]['Index']

        for trial in test_sequence:
            trial['Image_Index'] = test_image_lookup.get(trial['image_label'], None)

        # 保存test部分
        test_df = pd.DataFrame(test_sequence)
        test_df['Block'] = block
        test_df['Part'] = 'test'

        test_filename = os.path.join(sub_dir, f"block{block:02d}_test.csv")
        test_df.to_csv(test_filename, index=False)

        print(f"    已生成: {study_filename}")
        print(f"    已生成: {test_filename}")

print("所有被试的实验序列生成完成！")

为被试 1 生成实验序列...
  生成block 1...
    Block 1 使用的filler图片: ['human2.jpg', 'nonhuman7.jpg', 'nonhuman4.jpg', 'human3.jpg', 'nonhuman11.jpg']
    已生成: /Users/xuezhang/Documents/SKKU/ReconMem_new_design/Experiment/sub-01/block01_study.csv
    已生成: /Users/xuezhang/Documents/SKKU/ReconMem_new_design/Experiment/sub-01/block01_test.csv
  生成block 2...
    Block 2 使用的filler图片: ['nonhuman8.jpg', 'nonhuman1.jpg', 'nonhuman9.jpg', 'human2.jpg', 'nonhuman14.jpg']
    已生成: /Users/xuezhang/Documents/SKKU/ReconMem_new_design/Experiment/sub-01/block02_study.csv
    已生成: /Users/xuezhang/Documents/SKKU/ReconMem_new_design/Experiment/sub-01/block02_test.csv
  生成block 3...
    Block 3 使用的filler图片: ['nonhuman13.jpg', 'nonhuman15.jpg', 'nonhuman4.jpg', 'nonhuman9.jpg', 'nonhuman10.jpg']
    已生成: /Users/xuezhang/Documents/SKKU/ReconMem_new_design/Experiment/sub-01/block03_study.csv
    已生成: /Users/xuezhang/Documents/SKKU/ReconMem_new_design/Experiment/sub-01/block03_test.csv
  生成block 4...
    Block 4 使用的filler图